TODO pasar a funciones el cálculo de energía de gen 

In [ ]:
# Energy from the ME model

In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# load de ME model
with open('../files/models/iJL1678b_solver.pickle', 'rb') as f:
    me = pickle.load(f)

In [3]:
from energy_analisis import get_energy_consumption_production

In [4]:
# Get the consumption and production of energy per reaction
reactions_energy_consumption,reactions_energy_production = get_energy_consumption_production(me)

In [5]:
energy_consumption = pd.DataFrame.from_dict(reactions_energy_consumption).set_index('Reactions')
energy_consumption.head()

,Type,Total,Used,ATP
Reactions,,,,
transcription_TU0_8529_from_RpoD_mono,TranscriptionReaction,-347.0,-3.988197e-09,-347.0
transcription_TU0_8464_from_RpoD_mono,TranscriptionReaction,-88.0,-2.039023e-04,-88.0
transcription_TU0_13006_from_RpoD_mono,TranscriptionReaction,-172.0,-5.661036e-12,-172.0
transcription_TU0_5003_from_RPOH_MONOMER,TranscriptionReaction,-434.0,-1.200694e-07,-434.0
transcription_TU0_3423_from_RPOE_MONOMER,TranscriptionReaction,-109.0,-3.812990e-06,-109.0


In [19]:
energy_consumption.to_pickle('../files/energy/energy_per_reaction.pickle')

In [20]:
with open('../files/energy/energy_per_reaction.pickle', 'rb') as f:
    energy_consumption = pickle.load(f)

In [21]:
data = energy_consumption
data.loc[:,'Used'] = abs(data.Used)
elegidos=data
elegidos.head()

,Type,Total,Used,ATP
Reactions,,,,
transcription_TU0_8529_from_RpoD_mono,TranscriptionReaction,-347.0,3.988197e-09,-347.0
transcription_TU0_8464_from_RpoD_mono,TranscriptionReaction,-88.0,2.039023e-04,-88.0
transcription_TU0_13006_from_RpoD_mono,TranscriptionReaction,-172.0,5.661036e-12,-172.0
transcription_TU0_5003_from_RPOH_MONOMER,TranscriptionReaction,-434.0,1.200694e-07,-434.0
transcription_TU0_3423_from_RPOE_MONOMER,TranscriptionReaction,-109.0,3.812990e-06,-109.0


In [10]:
elegidos = elegidos[elegidos.Type=='MetabolicReaction']
genes_met={}
faltante_metabolico=0
elegidos_maximos={}
genes_tot=[]
reactant_tipos=[]
Complejos = []
complex_formation = []
for reaction in elegidos.index:
    elegidos_maximos[reaction]=[]
    a =me.reactions.get_by_id(reaction)
    for reactant in a.reactants:
        reactant_tipos.append(type(reactant).__name__)
        if((type(reactant).__name__)) =='Complex':
            Complejos.append(reactant)
            for reaccion_complejo in reactant.reactions:
                if (type(reaccion_complejo).__name__) == 'ComplexFormation':
                    complex_formation.append(reaccion_complejo)

                    n_genes=0
                    for reactant_complejo in reaccion_complejo.reactants:
                        # Chech how many genes are involved
                        if (type(reactant_complejo).__name__) == 'TranslatedGene':
#                             print(reaction,reactant_complejo,'\n')
                            elegidos_maximos[reaction].append(reactant_complejo.id.split('_')[1])
                            genes_tot.append(reactant_complejo.id.split('_')[1])
                            n_genes+=1
                    # Check each gene contribution 
                    if n_genes > 1:
                        total = 0
                        valores = {}
                        for reactant_complejo in reaccion_complejo.reactants:
                            if (type(reactant_complejo).__name__) == 'TranslatedGene':
                                                    valores[reactant_complejo.id] = abs(reaccion_complejo.get_coefficient(reactant_complejo))
                        total = np.sum(list(valores.values()))
    
                        for key,value in valores.items():
                            bnumber_m= key.split('_')[1]
                            genes_met[bnumber_m] = genes_met.get(bnumber_m, 0) + value/total*a.x
                    elif n_genes == 1:
                        genes_met[bnumber_m] = genes_met.get(bnumber_m, 0) + a.x
                    else:
                        faltante_metabolico+=(a.x)



In [55]:
pd.DataFrame(genes_met, index=['Metabolic']).T.head()


,Metabolic
b3255,0.016535
b2316,0.016859
b0185,0.016908
b3256,32.329844
b2751,0.093704


In [11]:
transcription = data.Used[data.Type=="TranscriptionReaction"]
genes_transcription={}
for ind in transcription.index:
    
    rxn = me.reactions.get_by_id(ind)
    num_gen= 0
    lista_genes = []
    for gene in rxn.products:
        
        if type(gene).__name__ == 'TranscribedGene':
            lista_genes.append(gene.id)
    if len(lista_genes) > 1:
                    total = 0
                    valores = {}
                    for gene in lista_genes:
                           valores[gene] = abs(rxn.get_coefficient(gene))
                            
                    total = np.sum(list(valores.values()))
    
                    for key,value in valores.items():
                        bnumber_m= key.split('_')[1]
                        genes_transcription[bnumber_m] = genes_transcription.get(bnumber_m, 0) + value/total*rxn.x
    elif len(lista_genes) == 1:
                        bnumber_m= lista_genes[0].split('_')[1]
                        genes_transcription[bnumber_m] = genes_transcription.get(bnumber_m, 0) + rxn.x
    else:
        print(rxn.x)

In [56]:
pd.DataFrame(genes_transcription, index=['Transcription']).T.head()



,Transcription
b0423,1.149336e-11
b3986,2.445798e-06
b0630,3.291300e-14
b4143,1.383288e-10
b4142,1.383288e-10


In [ ]:
elegidos_trad = data.loc[data.Type=='TranslationReaction']
elegidos_trad = elegidos_trad.Used

In [43]:
elegidos_trad.head()

Reactions
translation_b0003    0.010575
translation_b0004    0.000034
translation_b0014    0.000001
translation_b0015    0.000001
translation_b0023    0.007112
Name: Used, dtype: float64

In [35]:
totales = {}
for trans_bnumber in elegidos_trad.index:
    bnumber = trans_bnumber.split('_')[1]
    totales[bnumber] = genes_met.get(bnumber, 0) + elegidos_trad.loc[trans_bnumber] + genes_transcription.get(bnumber, 0)

In [41]:
Energia_total_ME = pd.DataFrame(genes_met.items(), columns=['Bnumber', 'Value'])
Energia_total_ME = Energia_total_ME.sort_values(by='Value',ascending=False).reset_index()
Energia_total_ME = Energia_total_ME.loc[:,['Bnumber','Value']]

In [18]:
Energia_total_ME.to_pickle('../files/energy/energy_per_gene.pickle')

In [28]:
Energia_total_ME.head()

,Bnumber,Value
0,b3256,32.329346
1,dummy,1.671430
2,b2752,0.977113
3,b1677,0.232180
4,b0727,0.194296
